In [1]:

# # Install a pip package in the current Jupyter kernel
# import sys
# !{sys.executable} -m pip install statsmodels
import pandas as pd
from collections import Counter
import numpy as np
from statsmodels.sandbox.stats.multicomp import multipletests


In [2]:
path_file = "D:/Hanze_Groningen/STAGE/analyse/stat/beforeGene_NonCoding_Coding_both_0_TESTS.tsv"
df = pd.read_csv(path_file, sep='\t')

In [3]:
df.columns

Index(['gene', 'chr', 'start_position_regio', 'end_position_regio',
       'counts_breast', 'counts_nonbreast', 'filter_snps_b', 'filter_snps_nb',
       'counts_breast_plus', 'counts_nonbreast_plus', 'X2', 'p_value_X2_self',
       'log10_p_value_X2_self', 'p_value_X2', 'log2_fc', 'p_value_F'],
      dtype='object')

In [4]:
p_value_X2_self = list(df['p_value_X2_self'])
p_value_X2 = list(df['p_value_X2'])
p_value_F = list(df['p_value_F'])

In [5]:
df_select = df[['gene', 'chr', 'start_position_regio', 'end_position_regio', 'counts_breast', 'counts_nonbreast',
                'p_value_X2_self', 'p_value_X2', 'p_value_F']]

In [6]:
bon_p_adjusted_value_X2_self = multipletests(p_value_X2_self, alpha=0.05, method='bonferroni')

In [7]:
# Create a list of the adjusted p-values --> one-step correction
bon_rej_X2_self, bon_p_adjusted_value_X2_self = multipletests(p_value_X2_self, alpha=0.05, method='bonferroni')[:2]
bon_rej_X2, bon_p_adjusted_value_X2 = multipletests(p_value_X2, alpha=0.05, method='bonferroni')[:2]
bon_rej_F, bon_p_adjusted_value_F = multipletests(p_value_F, alpha=0.05, method='bonferroni')[:2]
df_select['bon_X2_self'] = bon_p_adjusted_value_X2_self
df_select['bon_X2'] = bon_p_adjusted_value_X2
df_select['bon_F'] = bon_p_adjusted_value_F


C:\Users\Anne_\AppData\Local\Temp\ipykernel_2816\3978286935.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_select['bon_X2_self'] = bon_p_adjusted_value_X2_self
C:\Users\Anne_\AppData\Local\Temp\ipykernel_2816\3978286935.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_select['bon_X2'] = bon_p_adjusted_value_X2
C:\Users\Anne_\AppData\Local\Temp\ipykernel_2816\3978286935.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

In [8]:
# Create a list of the adjusted p-values --> Benjamini/Hochberg  (non-negative)
bh_rej_X2_self, bh_p_adjusted_value_X2_self = multipletests(p_value_X2_self, alpha=0.05, method='fdr_bh')[:2]
bh_rej_X2, bh_p_adjusted_value_X2 = multipletests(p_value_X2, alpha=0.05, method='fdr_bh')[:2]
bh_rej_F, bh_p_adjusted_value_F = multipletests(p_value_F, alpha=0.05, method='fdr_bh')[:2]
df_select['bh_X2_self'] = bh_p_adjusted_value_X2_self
df_select['bh_X2'] = bh_p_adjusted_value_X2
df_select['bh_F'] = bh_p_adjusted_value_F

#??? 'fdr_tsbh' : two stage fdr correction (Benjamini/Hochberg)
#`fdr_i`, `fdr_bh` : Benjamini/Hochberg  (non-negative)

C:\Users\Anne_\AppData\Local\Temp\ipykernel_2816\2864826296.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_select['bh_X2_self'] = bh_p_adjusted_value_X2_self
C:\Users\Anne_\AppData\Local\Temp\ipykernel_2816\2864826296.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_select['bh_X2'] = bh_p_adjusted_value_X2
C:\Users\Anne_\AppData\Local\Temp\ipykernel_2816\2864826296.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_i

In [9]:
df_select

,gene,chr,start_position_regio,end_position_regio,counts_breast,counts_nonbreast,p_value_X2_self,p_value_X2,p_value_F,bon_X2_self,bon_X2,bon_F,bh_X2_self,bh_X2,bh_F
0,DDX11L1,1,9873,12123,1,5,0.771825,1.000000,0.559154,1.0,1.0,1.0,0.938073,1.0,1.0
1,WASH7P,1,12361,14611,0,37,0.020144,0.035909,0.010904,1.0,1.0,1.0,0.619264,1.0,1.0
2,WASH7P,1,12406,14656,0,38,0.018529,0.032903,0.011297,1.0,1.0,1.0,0.585982,1.0,1.0
3,FAM138A,1,32610,34860,0,0,0.887706,1.000000,1.000000,1.0,1.0,1.0,0.938073,1.0,1.0
4,OR4F5,1,67090,69340,0,1,0.709454,1.000000,1.000000,1.0,1.0,1.0,0.937990,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31843,LAMA4,6,112427133,112429383,1,8,0.884413,1.000000,1.000000,1.0,1.0,1.0,0.938073,1.0,1.0
31844,Mir_548,6,112588827,112591077,1,7,0.984820,1.000000,1.000000,1.0,1.0,1.0,0.984820,1.0,1.0
31845,RFPL4B,6,112666531,112668781,0,8,0.281121,0.581742,0.606698,1.0,1.0,1.0,0.824540,1.0,1.0
31846,MARCKS,6,114176526,114178776,1,4,0.625170,1.000000,0.494591,1.0,1.0,1.0,0.892603,1.0,1.0


In [10]:
df_X2_self = df_select[['gene', 'p_value_X2_self', 'bon_X2_self', 'bh_X2_self']]
df_X2 = df_select[['gene', 'p_value_X2', 'bon_X2', 'bh_X2']]
df_F = df_select[['gene', 'p_value_F', 'bon_F', 'bh_F']]



In [12]:
def get_significant(df, test_type):
    print(f'---------{test_type}--------')
    normal_p = df[df[f'p_value_{test_type}'] >= 0.05]
    sort_normal_p = normal_p.sort_values(f'p_value_{test_type}')
    genes_normal = sort_normal_p['gene']
    print(f'nomal: {len(genes_normal)}')

    bon_p = df[df[f'bon_{test_type}'] >= 0.05]
    sort_bon = bon_p.sort_values(f'bon_{test_type}')
    bon_genes = sort_bon['gene']
    print(f'bon: {len(bon_genes)}')

    bh_p = df[df[f'bh_{test_type}'] >= 0.05]
    sort_bh = bh_p.sort_values(f'bh_{test_type}')
    bh_genes = sort_bh['gene']
    print(f'bh: {len(bh_genes)}')


    return genes_normal, bon_genes, bh_genes
    

In [13]:
X2_self_genes_normal, X2_self_bon_genes, X2_self_bh_genes = get_significant(df_X2_self, 'X2_self')
X2_genes_normal, X2_bon_genes, X2_bh_genes = get_significant(df_X2, 'X2')
F_genes_normal, F_bon_genes, F_bh_genes = get_significant(df_F, 'F')


---------X2_self--------
nomal: 30311
bon: 31818
bh: 31664
---------X2--------
nomal: 31145
bon: 31833
bh: 31797
---------F--------
nomal: 30775
bon: 31834
bh: 31817


In [17]:
def print_statements(self_X2_df, X2_df, F_df):
    print('---ALL')
    all_values = set(self_X2_df) | set(X2_df) |  set(F_df)
    print(f'All genes ({len(all_values)}): {all_values}')
    elements_in_all = list(set.intersection(*map(set, [self_X2_df, X2_df, F_df])))
    print(f'Gene in all list ({len(elements_in_all)}): {elements_in_all}\n')
    # selfX2 and X2
    print('---selfX2 and X2')
    in_selfX2_X2 = set(self_X2_df) & set(X2_df)
    print(f'in selfX2 and X2 ({len(in_selfX2_X2)}): {in_selfX2_X2}')
    dif_selfX2_X2 = set(self_X2_df) - set(X2_df)
    print(f'in selfX2 NOT in X2 ({len(dif_selfX2_X2)}): {dif_selfX2_X2}')
    dif_X2_selfX2 = set(X2_df) - set(self_X2_df)
    print(f'in X2 NOT in selfx2 ({len(dif_X2_selfX2)}): {dif_X2_selfX2}\n')
    # selfX2 and F
    print('---selfX2 and F')
    in_selfX2_F = set(self_X2_df) & set(F_df)
    print(f'in selfX2 and F ({len(in_selfX2_F)}): {in_selfX2_F}')
    dif_selfX2_F = set(self_X2_df) - set(F_df)
    print(f'in selfX2 NOT in F ({len(dif_selfX2_F)}): {dif_selfX2_F}')
    dif_F_selfX2 = set(F_df) - set(self_X2_df)
    print(f'in F NOT in selfX2 ({len(dif_F_selfX2)}): {dif_F_selfX2}\n')
    # F and X2
    print('---F and X2')
    in_F_X2 = set(F_df) & set(X2_df)
    print(f'in F and X2 ({len(in_F_X2)}): {in_F_X2}')
    dif_F_X2 = set(F_df) - set(X2_df)
    print(f'in F NOT in X2 ({len(dif_F_X2)}): {dif_F_X2}')
    dif_X2_F = set(X2_df) - set(F_df)
    print(f'in X2 NOT in F ({len(dif_X2_F)}): {dif_X2_F}')

In [18]:
top_num = 20

In [19]:
print_statements(X2_self_genes_normal[:top_num], X2_genes_normal[:top_num], F_genes_normal[:top_num])


---ALL
All genes (60): {'MAS1L', 'MIR127', 'MIR148B', 'AGER', 'RP11-457M11.5', 'LCE3D', 'DELEC1', 'MIR136', 'Mir_548', 'MYL6', 'HEATR1', 'STX16', 'MIR548AR', 'WNT2B', 'Y_RNA', 'BC042029', 'SLC16A7', 'ZNF716', 'DQ571461', 'MIR4758', 'IQCG', 'KRTAP4-7', 'FAM106C', 'DQ596206', 'LINC01070', 'PRIM1', 'OR6K2', 'MIR3939', 'TRNA_Gln', 'CXCL12', 'TMPRSS9', 'TRNA_Cys', 'PLEKHG5', 'ZNF692', 'KRTAP9-6', 'PUS3', 'ITPRID2', 'CHPF', 'SLC35F3', 'EOLA1', 'RPL35', 'NCBP2AS2', 'SLC25A42', 'MIR548AQ', 'BC042823', 'FGFRL1', 'RP11-637A17.2', 'FAM131C', 'MIR432', 'OR4A16', 'LINC02453', 'H2BC9', 'CTU2', 'BC071797', 'MIR3689D2', 'PTPRR', 'BC045791', 'ARHGEF39', 'ITGAD', 'BBIP1'}
Gene in all list (0): []

---selfX2 and X2
in selfX2 and X2 (0): set()
in selfX2 NOT in X2 (20): {'DELEC1', 'HEATR1', 'MIR548AR', 'Y_RNA', 'ZNF716', 'MIR4758', 'IQCG', 'KRTAP4-7', 'TRNA_Gln', 'CXCL12', 'TMPRSS9', 'PUS3', 'CHPF', 'RPL35', 'MIR548AQ', 'FAM131C', 'OR4A16', 'PTPRR', 'BC045791', 'BBIP1'}
in X2 NOT in selfx2 (20): {'MIR148B'

In [20]:
print_statements(X2_self_bon_genes[:top_num], X2_bon_genes[:top_num], F_bon_genes[:top_num])


---ALL
All genes (56): {'ACR', 'SMARCA1', 'MYL9', 'ABHD14A', 'FER1L4', 'NPHS1', 'NEK8', 'AAR2', 'APLP1', 'SEPTIN9', 'RP11-234K24.3', 'MIR223HG', 'OR4C3', 'MIR190B', 'ITGB3BP', 'DQ576627', 'TRBV10-1', 'DLGAP4', 'ZNF703', 'RNU7-171P', 'ERAL1', 'AC005048.1', 'GPR137C', 'CYB5A', 'NOS2', 'RNU2-42P', 'CCDC144NL-AS1', 'TM4SF18', 'XXbac-B476C20.9', 'SNORA51', 'GRB7', 'XBP1', 'LOC642236', 'S100A14', 'TRBV5-4', 'PMFBP1', 'LINC01410', 'MUC12', 'SRCAP', 'OR8U8', 'WDR44', 'RP11-146F11.1', 'MIR633', 'L37717', 'CNPY4', 'ATP6V0E2-AS1', 'IFT88', 'RP11-280G9.1', 'TRNA_Pseudo', 'DYNLT3', 'APEX2', 'DLGAP4-AS1', 'MIR3613', 'TIMM44', 'ZNF891', 'IFNGR2'}
Gene in all list (0): []

---selfX2 and X2
in selfX2 and X2 (3): {'RP11-280G9.1', 'MIR633', 'ZNF703'}
in selfX2 NOT in X2 (16): {'PMFBP1', 'IFT88', 'RP11-146F11.1', 'AC005048.1', 'ABHD14A', 'MIR3613', 'NOS2', 'TRNA_Pseudo', 'ZNF891', 'NPHS1', 'SEPTIN9', 'WDR44', 'XBP1', 'TIMM44', 'S100A14', 'IFNGR2'}
in X2 NOT in selfx2 (17): {'ITGB3BP', 'SMARCA1', 'RNU7-171

In [21]:
print_statements(X2_self_bh_genes[:top_num], X2_bh_genes[:top_num], F_bh_genes[:top_num])


---ALL
All genes (60): {'EBP', 'SMARCA1', 'TRNA_Asn', 'RP5-1171I10.5', 'LOC100499194', 'NEK8', 'MAGEB10', 'JB025504', 'AKAP17A', 'Y_RNA', 'ITGB3BP', 'LILRB5', 'RP4-738P15.1', 'RNU7-171P', 'ERAL1', 'ZNF703', 'AC005048.1', 'MIR4273', 'LOC440461', 'CYB5A', 'OR4C45', 'DLG1-AS1', 'CCDC144NL-AS1', 'TRBV3-1', 'C1orf100', 'MIR663B', 'TM4SF18', 'MID1IP1', 'GRB7', 'DDX39A', 'PPP1R1B', 'S100A14', 'SNAPC2', 'TRBV5-4', 'ZNF569', 'PMFBP1', 'LINC01410', 'HDAC8', 'FRG1', 'SRCAP', 'AOC4', 'L37717', 'AC002310.7', 'RP11-146F11.1', 'ASCC2', 'VHLL', 'FN3K', 'MIR633', 'ALDH5A1', 'JA700154', 'IFT88', 'PMCHL2', 'RP11-280G9.1', 'DYNLT3', 'APEX2', 'ACAD10', 'ARMCX5', 'MICOS13', 'DD157417', 'KIF12'}
Gene in all list (0): []

---selfX2 and X2
in selfX2 and X2 (0): set()
in selfX2 NOT in X2 (20): {'TRNA_Asn', 'LOC100499194', 'Y_RNA', 'LILRB5', 'RP4-738P15.1', 'LOC440461', 'DLG1-AS1', 'KIF12', 'MID1IP1', 'DDX39A', 'ZNF569', 'HDAC8', 'L37717', 'ASCC2', 'FN3K', 'ALDH5A1', 'JA700154', 'PMCHL2', 'ACAD10', 'ARMCX5'}
in 